# imports

In [17]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate

# loading llm model: llama

## llm

In [4]:
llm = Ollama(model="llama3:8b", temperature=0)

In [5]:
text = "what would be a good company name for a company that makes colorful socks ?"

print(llm.invoke(text))

What a fun question!

Here are some ideas for a company name that might fit the bill:

1. **SockItToMe**: A playful name that references the idea of putting your best foot forward (or in this case, your most colorful sock).
2. **HueHub**: A nod to the vibrant colors and hues you'd find in a pair of colorful socks.
3. **ToeTally Fun**: A whimsical name that captures the lighthearted spirit of wearing fun, colorful socks.
4. **SoleMates**: A clever play on words that references the idea of finding your perfect match (in this case, a pair of matching colorful socks).
5. **KaleidoKrew**: A name that references the kaleidoscope of colors and patterns you might find in a pair of colorful socks.
6. **SockSational**: A playful name that emphasizes the excitement and joy of wearing colorful socks.
7. **FiberFrenzy**: A name that highlights the creative, artistic aspect of designing and making colorful socks.
8. **HeelToToe**: A clever name that references the entire sock, from heel to toe (and 

## chat model

In [6]:
chat_model = ChatOllama(model="llama3:8b", temperature=0)

In [9]:
text = "what would be a good company name for a company that makes colorful socks ?"
messages = [HumanMessage(content=text)]

response = chat_model.invoke(input=messages)

In [13]:
print(type(response))
response.dict()

<class 'langchain_core.messages.ai.AIMessage'>


{'content': "What a fun question!\n\nHere are some ideas for a company name that might fit the bill:\n\n1. **SockItToMe**: A playful name that references the idea of wearing your favorite socks to show them off.\n2. **HueHub**: A nod to the colorful hues of your socks, with a fun and modern twist.\n3. **ToeTally Colorful**: A whimsical name that captures the playful spirit of your products.\n4. **SoleMates Socks**: A clever reference to the idea of finding your perfect match (in this case, a pair of colorful socks).\n5. **Fiber & Fancy**: A name that highlights the quality and style of your socks, with a touch of whimsy.\n6. **SockSational**: A fun, alliterative name that conveys the excitement and joy of wearing colorful socks.\n7. **Kaleidoscope Krew**: A name that references the colorful, swirling patterns often found in your socks.\n8. **StepUp Socks**: A motivational name that encourages people to step up their sock game with your vibrant, colorful designs.\n9. **SockSavvy**: A cl

# prompt template

In [22]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product} ?")
prompt.format(product= "colorful socks")

'What is a good name for a company that makes colorful socks ?'

In [23]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant that translates {input_language} to {output_language}."),
    ("human", "{text}")
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='you are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]